In [1]:
pip install spacy

  Using cached spacy-3.7.6-cp312-cp312-win_amd64.whl.metadata (27 kB)
  Using cached preshed-3.0.9-cp312-cp312-win_amd64.whl.metadata (2.2 kB)
  Using cached thinc-8.2.5-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached srsly-2.4.8-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached typer-0.12.5-py3-none-any.whl.metadata (15 kB)
  Using cached langcodes-3.4.1-py3-none-any.whl.metadata (29 kB)
  Using cached language_data-1.2.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached blis-0.7.11-cp312-cp312-win_amd64.whl.metadata (7.6 kB)
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
Using cached spacy-3.7.6-cp312-cp312-win_amd64.whl (11.7 MB)
Using cached catalogue-2.0.10-py3-none-any.whl (17 kB)
Using cached langcodes-3.4.1-py3-none-any.whl (182 kB)
Using cached preshed-3.0.9-cp312-cp312-win_amd64.whl (122 kB)
Using cached srsly-2.

In [1]:
!python -m spacy download en_core_web_md


     ---------------------------------------- 0.0/42.8 MB ? eta -:--:--
     --------------------------------------- 0.0/42.8 MB 660.6 kB/s eta 0:01:05
     --------------------------------------- 0.1/42.8 MB 656.4 kB/s eta 0:01:06
     --------------------------------------- 0.1/42.8 MB 655.4 kB/s eta 0:01:06
     --------------------------------------- 0.1/42.8 MB 655.4 kB/s eta 0:01:06
     --------------------------------------- 0.2/42.8 MB 706.2 kB/s eta 0:01:01
     --------------------------------------- 0.2/42.8 MB 706.2 kB/s eta 0:01:01
     --------------------------------------- 0.2/42.8 MB 706.2 kB/s eta 0:01:01
     --------------------------------------- 0.2/42.8 MB 706.2 kB/s eta 0:01:01
     --------------------------------------- 0.2/42.8 MB 706.2 kB/s eta 0:01:01
     --------------------------------------- 0.2/42.8 MB 706.2 kB/s eta 0:01:01
     --------------------------------------- 0.2/42.8 MB 706.2 kB/s eta 0:01:01
     --------------------------------------- 0.2

In [ ]:
import random
import speech_recognition as sr
import pyttsx3
import time
import cv2
import threading
import numpy as np
import datetime
from fuzzywuzzy import process  # Import fuzzywuzzy for matching
import spacy

# Load the spaCy model for English
nlp = spacy.load("en_core_web_md")

# Initialize recognizer and text-to-speech engine
recognizer = sr.Recognizer()
tts_engine = pyttsx3.init()

# Define the career options dataset with keywords for validation
career_options = {
    "Software Engineer": [
        {"question": "Can you explain the difference between object-oriented and functional programming?",
         "keywords": ["object-oriented", "functional", "classes", "functions", "encapsulation", "immutability"]},
        {"question": "What are your favorite programming languages and why?",
         "keywords": ["Python", "Java", "C++", "JavaScript", "features", "libraries", "community"]},
        {"question": "Describe a challenging bug you faced and how you resolved it.",
         "keywords": ["bug", "debugging", "tools", "fix", "solution"]},
    ],
    "Data Scientist": [
        {"question": "What is the difference between supervised and unsupervised learning?",
         "keywords": ["supervised", "unsupervised", "machine learning", "training", "data"]},
        {"question": "Explain how you would handle missing data in a dataset.",
         "keywords": ["missing data", "imputation", "deletion", "data cleaning"]},
        {"question": "Describe a project where you used statistical analysis.",
         "keywords": ["statistical analysis", "project", "data", "insights", "tools"]},
    ],
    "Project Manager": [
        {"question": "How do you define project success?",
         "keywords": ["success", "goals", "objectives", "stakeholders", "deliverables"]},
        {"question": "What project management methodologies are you familiar with?",
         "keywords": ["agile", "waterfall", "scrum", "Kanban", "methodologies"]},
        {"question": "How do you handle conflicts within your team?",
         "keywords": ["conflict", "resolution", "communication", "mediation"]},
    ],
    "DevOps Engineer": [
        {"question": "What are some of the key tools used in DevOps?",
         "keywords": ["Jenkins", "Docker", "Kubernetes", "Ansible", "automation", "CI/CD"]},
        {"question": "What is the importance of monitoring in DevOps?",
         "keywords": ["monitoring", "logs", "metrics", "alerts", "uptime"]},
        {"question": "How would you ensure continuous integration in a development pipeline?",
         "keywords": ["integration", "automation", "testing", "pipeline", "build"]},
    ],
    "Cybersecurity Specialist": [
        {"question": "Can you explain the differences between symmetric and asymmetric encryption?",
         "keywords": ["symmetric", "asymmetric", "encryption", "keys", "security"]},
        {"question": "What are the common cybersecurity threats?",
         "keywords": ["malware", "phishing", "ransomware", "DDoS", "vulnerabilities"]},
        {"question": "How would you secure a company's network infrastructure?",
         "keywords": ["firewall", "VPN", "intrusion detection", "encryption", "policies"]},
    ],
    "UX/UI Designer": [
        {"question": "What is your design process when starting a new project?",
         "keywords": ["design process", "research", "prototyping", "user testing"]},
        {"question": "How do you ensure your designs are user-friendly?",
         "keywords": ["user experience", "usability", "accessibility", "feedback"]},
        {"question": "Can you explain the importance of user personas in design?",
         "keywords": ["user personas", "target audience", "design decisions", "user needs"]},
    ],
    "Business Analyst": [
        {"question": "What techniques do you use to gather requirements?",
         "keywords": ["gather requirements", "interviews", "surveys", "stakeholders"]},
        {"question": "How do you prioritize requirements?",
         "keywords": ["prioritize", "criteria", "stakeholders", "business value"]},
        {"question": "Describe a time when you solved a complex business problem.",
         "keywords": ["problem-solving", "analysis", "solution", "impact"]},
    ],
    "Cloud Engineer": [
        {"question": "What cloud platforms are you familiar with?",
         "keywords": ["AWS", "Azure", "Google Cloud", "cloud computing", "services"]},
        {"question": "How do you approach cloud security?",
         "keywords": ["cloud security", "encryption", "access control", "best practices"]},
        {"question": "Can you describe a cloud migration project you've worked on?",
         "keywords": ["cloud migration", "project", "planning", "execution"]},
    ],
    "Marketing Specialist": [
        {"question": "What marketing channels do you consider most effective?",
         "keywords": ["marketing channels", "social media", "email", "SEO", "content marketing"]},
        {"question": "How do you measure the success of a marketing campaign?",
         "keywords": ["metrics", "KPIs", "analytics", "ROI"]},
        {"question": "Describe a successful marketing strategy you implemented.",
         "keywords": ["strategy", "execution", "results", "impact"]},
    ],
    "Data Analyst": [
        {"question": "What tools do you use for data analysis?",
         "keywords": ["Excel", "SQL", "Tableau", "Python", "data visualization"]},
        {"question": "How do you ensure data accuracy and integrity?",
         "keywords": ["data accuracy", "validation", "cleaning", "quality checks"]},
        {"question": "Can you explain a data-driven decision you made?",
         "keywords": ["data-driven", "decision-making", "insights", "impact"]},
    ],
}

# Function to convert text to speech and print
def speak_text(text):
    print(text)  # Print to Jupyter Notebook output
    tts_engine.say(text)
    tts_engine.runAndWait()

# Function to get user input via speech with a 2-minute time limit
def get_speech_input(prompt, time_limit=120):
    speak_text(prompt)
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source)
        print("Listening...")
        start_time = time.time()
        audio = None

        while True:
            try:
                if time.time() - start_time > time_limit:
                    print("Time's up!")
                    speak_text("Time's up!")
                    return ""
                audio = recognizer.listen(source, timeout=5)

                # Try recognizing the speech
                text = recognizer.recognize_google(audio)
                print(f"Recognized speech: {text}")
                return text
            except sr.UnknownValueError:
                speak_text("Sorry, I could not understand your speech. Please try again.")
            except sr.WaitTimeoutError:
                speak_text("No speech detected.")
            except sr.RequestError as e:
                speak_text("Sorry, there was an error with the speech recognition service.")
                return None

# Function to calculate semantic similarity score
def calculate_semantic_score(question, answer):
    question_doc = nlp(question)
    answer_doc = nlp(answer)
    
    # Get the similarity score between the question and the answer
    similarity_score = question_doc.similarity(answer_doc)
    
    # Scale it to a score out of 10 (or any other scale you prefer)
    return similarity_score * 10  # This gives a score from 0 to 10

# Function to start camera recording
def start_camera(cap, out, current_question):
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Overlay the current question on the video frame
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, current_question[0], (10, 50), font, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Write the frame to the video file
        out.write(frame)

        # Display the video frame
        cv2.imshow('Interview Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Function to conduct the interview
def conduct_interview():
    # Try different camera indexes
    for camera_index in range(0, 3):
        cap = cv2.VideoCapture(camera_index)
        if cap.isOpened():
            print(f"Camera opened successfully with index {camera_index}")
            break
        else:
            cap.release()
    else:
        print("Error: Could not open any video device.")
        speak_text("Error: Could not open any video device.")
        return

    # Get the current timestamp for the video file name
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    video_filename = f"interview_{timestamp}.avi"

    # Define the codec and create a VideoWriter object to save the video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(video_filename, fourcc, 20.0, (640, 480))

    total_score = 0
    current_question = [""]

    # Start the camera in a separate thread
    camera_thread = threading.Thread(target=start_camera, args=(cap, out, current_question))
    camera_thread.start()

    # Ask the user to choose a career option
    career_choice = ""
    while career_choice not in career_options:
        speak_text("Please choose a career option from the following: ")
        print("Available career options:")
        for option in career_options.keys():
            print(option)
            speak_text(option)
        career_choice = input("Enter your choice: ").strip()

    questions = career_options[career_choice]
    random.shuffle(questions)  # Shuffle questions for variety

    for item in questions:
        question = item["question"]

        # Speak the question and update the current question for display
        speak_text(question)
        print(f"Question: {question}")  # Print question to notebook output
        current_question[0] = question
        answer = get_speech_input("Your answer: ", time_limit=2)

        if answer:
            # Use semantic scoring instead of keyword validation
            score = calculate_semantic_score(question, answer)
            total_score += score
            speak_text(f"You scored {score:.2f} for this answer.")
            print(f"You scored {score:.2f} for this answer.")  # Print to notebook output
        else:
            speak_text("No valid answer received.")
            print("No valid answer received.")  # Print to notebook output

    # Cleanup
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    camera_thread.join()  # Wait for the camera thread to finish

    # Final score
    speak_text(f"Your total score is {total_score:.2f}. Thank you for participating!")
    print(f"Your total score is {total_score:.2f}. Thank you for participating!")  # Print to notebook output

# Start the interview process
conduct_interview()


E:\anaconda\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Camera opened successfully with index 0
Please choose a career option from the following: 
Available career options:
Software Engineer
Software Engineer
Data Scientist
Data Scientist
Project Manager
Project Manager
DevOps Engineer
DevOps Engineer
Cybersecurity Specialist
Cybersecurity Specialist
UX/UI Designer
UX/UI Designer
Business Analyst
Business Analyst
Cloud Engineer
Cloud Engineer
Marketing Specialist
Marketing Specialist
Data Analyst
Data Analyst


Enter your choice:  Marketing Specialist


What marketing channels do you consider most effective?
Question: What marketing channels do you consider most effective?
Your answer: 
Listening...
Sorry, I could not understand your speech. Please try again.
Time's up!
Time's up!
No valid answer received.
No valid answer received.
Describe a successful marketing strategy you implemented.
Question: Describe a successful marketing strategy you implemented.
Your answer: 
Listening...
Sorry, I could not understand your speech. Please try again.
Time's up!
Time's up!
No valid answer received.
No valid answer received.
How do you measure the success of a marketing campaign?
Question: How do you measure the success of a marketing campaign?
Your answer: 
Listening...
Sorry, I could not understand your speech. Please try again.
Time's up!
Time's up!
No valid answer received.
No valid answer received.
